In [66]:
import hddm
import os
import sys
import copy
import seaborn as sns
import pandas as pd
import glob
import matplotlib.pyplot as plt
import scipy.stats
from kabuki.analyze import gelman_rubin
from kabuki.analyze import check_geweke
import kabuki
print (os.environ['CONDA_DEFAULT_ENV'])
%matplotlib inline

hddm92


In [2]:
#sys.path
hddm.__version__ 

'0.9.5'

In [3]:
os.chdir('/home/despo/iballard/')
real_data = hddm.load_csv('/home/despo/rule_habit/hddm_data_full_center.csv')
real_data = real_data[real_data['cond'] == 'reward']
n_subjects = 1#len(set(real_data['sub']))

In [4]:
sample = real_data[real_data['sub'] == 'subj13']
sample['counter'] = 1
sample.groupby('miniblock_type').sum()

/home/despo/iballard/anaconda3/envs/hddm92/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,rt,response,coherence,coherence_bin,subj_idx,coherence_float,rt_flip,coherence_center,counter
miniblock_type,,,,,,,,,,
compete,481731,247.206940,108,109.12,284,564,284.0,37.458034,1.312741,188
noncompete,478630,246.664336,119,108.48,281,561,281.0,64.196867,-0.026372,187
rewarded,470679,236.501654,133,14.80,278,552,278.0,102.919379,1.322871,184


In [5]:
coherence = list(set(real_data['coherence_center']))
coherence.sort()
coherence

[-1.3441776374896397,
 -0.44974757815494015,
 0.4446824811797591,
 1.3391125405144584]

In [6]:
v_intercept = 0.827654
v_coherence_center = 0.425295
a_intercept = 1.619
t_intercept = 0.739154

a_compete = 0.00488717
a_reward = 0.024066

v_compete = -0.0181477
v_reward = 0.0920351

t_compete = 0.0102455
t_reward = 0.000698769

In [7]:
trials_per_level = 192 /4 #trials per condition for each coherence level

all_data = []
for c in coherence:

    #Next we set up parameters of the drift diffusion process for the three levels and the first stimulus. As desribed earlier v and z change accross levels


    noncompete = {'v':v_intercept + c*v_coherence_center, 
                  'a':a_intercept, 
                  't':t_intercept, 
                  'sv':0, 'z':.5, 'sz':0, 'st':0}

    compete = {'v':v_intercept + c*v_coherence_center + v_compete, 
               'a':a_intercept + a_compete,
               't':t_intercept + t_compete,
               'sv':0, 'z':.5, 'sz':0, 'st':0}

    reward = {'v':v_intercept + c*v_coherence_center + v_reward, 
              'a':a_intercept + a_reward, 
              't':t_intercept + t_reward, 
              'sv':0, 'z':.5, 'sz':0, 'st':0}

    #Now we generate the data for stimulus A
    data_a, params_a = hddm.generate.gen_rand_data({'noncompete': noncompete,
                                                    'compete': compete,
                                                    'reward': reward},
                                                    size=trials_per_level,
                                                    subjs=n_subjects)
    
    data_a['coherence'] = c
    
    all_data.append(data_a.copy())
all_data = pd.concat(all_data)

In [8]:
# all_data.to_csv('/home/despo/rule_habit/hddm_data_full_simulated.csv')

In [13]:
all_data.shape

(576, 5)

In [102]:
all_data = hddm.utils.flip_errors(all_data)


In [104]:
# burn = 500
# samples = 1000
# m = hddm.HDDMRegressor(all_data, ['v ~ coherence + C(condition, Treatment("noncompete"))',
#     							  'a ~ C(condition, Treatment("noncompete"))',
#     							  't ~ C(condition, Treatment("noncompete"))'],
#                            p_outlier=.05)
# m.find_starting_values()
# m.sample(samples, burn=burn)

In [70]:
m.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
v_Intercept,0.796708,0.059905,0.736742,0.745119,0.763805,0.859738,0.878135,0.0267903
"v_C(condition, Treatment(""noncompete""))[T.compete]",10.6021,9.46198,-1.67266,7.42316,8.21454,11.712,27.3334,4.23153
"v_C(condition, Treatment(""noncompete""))[T.reward]",5.62169,6.51208,-3.98986,0.492126,7.05772,11.0981,13.4504,2.91229
v_coherence,-0.047634,0.0251685,-0.0702699,-0.063372,-0.0561609,-0.0490297,0.000662437,0.0112557
a_Intercept,1.58519,0.0170233,1.55809,1.57462,1.5885,1.6019,1.60286,0.00761303
"a_C(condition, Treatment(""noncompete""))[T.compete]",-7.86107,9.29272,-17.6964,-15.2579,-11.9504,-1.46072,7.06004,4.15583
"a_C(condition, Treatment(""noncompete""))[T.reward]",-5.94735,9.83054,-23.5679,-6.77056,-4.64808,-0.794339,6.04411,4.39635
t_Intercept,0.736232,0.00200386,0.733346,0.735077,0.735834,0.738003,0.738901,0.000896153
"t_C(condition, Treatment(""noncompete""))[T.compete]",1.73076,7.83363,-8.36569,-3.48484,-1.30366,10.6008,11.2072,3.5033
"t_C(condition, Treatment(""noncompete""))[T.reward]",0.158019,10.235,-10.6149,-9.2633,-2.63881,6.81407,16.493,4.57722


In [10]:
nchains = 5
chains = []
for i in range(nchains):
    chain = hddm.load('./models/reward_block_a_v_t_thin_simulated_chain_' + str(i))
    chains.append(chain)
    print(i)

0
1
2
3
4


In [11]:
combined_model = kabuki.utils.concat_models(chains)

In [55]:
original = {'v_intercept' : 0.827654,
'v_coherence_center' : 0.425295,
'a_intercept' : 1.619,
't_intercept' : 0.739154,
'a_compete' : 0.00488717,
'a_reward' : 0.024066,
'v_compete' : -0.0181477,
'v_reward' : 0.0920351,
't_compete' : 0.0102455,
't_reward' : 0.000698769}
original = pd.DataFrame(original, index = [0])
# original['parameters'] = 'veridical'

original.columns.name = 'parameter'
original = original.stack()
original.name = 'value'
original = original.reset_index()
original = original.sort_values(by = ['parameter'])
original['parameters'] = 'veridical'

In [56]:
new = {'v_intercept' : 0.787178,
'v_coherence_center' : 0.376946,
'v_compete' : -0.000243889,
'v_reward' : 0.0810647,
'a_intercept' : 1.70509,
'a_compete' : -0.00641649,
'a_reward' : 0.0396683,
't_intercept' : 0.665747,
't_compete' : 0.0123109,
't_reward' : -0.00213291}
new = pd.DataFrame(new, index = [0])

new.columns.name = 'parameter'
new = new.stack()
new.name = 'value'
new = new.reset_index()
new['parameters'] = 'recovered'
new = new.sort_values(by = ['parameter'])
params = pd.concat([original,new])

In [69]:
diff = new['value'].values - original['value'].values
diff = [x**2 for x in diff]
print(sum(diff))

scipy.stats.pearsonr(new['value'],
                    original['value'])

0.017600393028288353


(0.9971109517958047, 3.037321573367998e-10)

In [80]:
import numpy as np
diff = new['value'].values - original['value'].values

diff = [x**2 for x in diff]

np.mean([100*d/x for d,x in zip(diff,original['value'].values)])

0.5113952530880284

In [62]:
original

,level_0,parameter,value,parameters
4,0,a_compete,0.004887,veridical
2,0,a_intercept,1.619000,veridical
5,0,a_reward,0.024066,veridical
8,0,t_compete,0.010245,veridical
3,0,t_intercept,0.739154,veridical
9,0,t_reward,0.000699,veridical
1,0,v_coherence_center,0.425295,veridical
6,0,v_compete,-0.018148,veridical
0,0,v_intercept,0.827654,veridical
7,0,v_reward,0.092035,veridical
